In [1]:
from utils.dataset_loader import dataset_loader


_, _, test_data_p = dataset_loader('cora')

In [5]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="dataset", name="Cora", split="full")
dataset = dataset[0]  # there is only one graph in the dataset

test_data = dataset.subgraph(dataset.test_mask)

for i in range(len(test_data.y)):
	print(test_data.y[i], test_data_p.y[i])

tensor(3) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(0) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(3) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(1) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2

In [1]:
from torch_geometric.datasets import Planetoid
import os
from torch_geometric.loader import DataLoader
import torch_geometric
import torch
import torch.nn.functional as F
from utils.dataset_loader import dataset_loader


DATASET_STORAGE_PATH = "./dataset/"

train_data, val_data, test_data = dataset_loader('cora')



In [16]:
# count the how many elements are for each class (are 1-hot encoded)
print(test_data.y[:,0].sum())
print(test_data.y[:,1].sum())
print(test_data.y[:,2].sum())
print(test_data.y[:,3].sum())


tensor(149.)
tensor(103.)
tensor(64.)
tensor(684.)


In [5]:
from utils.math import interval_softmax, actually_reachable
import torch
import numpy as np

a_L = np.array([9.6062404e-06, 9.9997389e-01, 1.6473108e-05])
a_U = np.array([9.6254726e-06, 9.9997395e-01, 1.6506086e-05])

assert np.all(a_L <= a_U), "Lower bound should be less than upper bound"

q_L, q_U = actually_reachable(a_L, a_U)

assert np.all(q_L <= q_U), "Lower bound should be less than upper bound"

print("Lower bound: ", q_L)
print("Upper bound: ", q_U)

Lower bound:  [9.6062404e-06 9.9997389e-01 1.6473108e-05]
Upper bound:  [9.62547260e-06 9.99973921e-01 1.65037596e-05]
